Documntation : https://openpyxl.readthedocs.io/en/3.1.3/tutorial.html
Guide : https://realpython.com/openpyxl-excel-spreadsheets-python/

In [1]:
import os
import subprocess

import openpyxl as opxl
from openpyxl.styles import Font, Color, Alignment, Border, Side
from openpyxl.styles import NamedStyle
from openpyxl.styles import PatternFill
from openpyxl.chart import PieChart, Reference
from openpyxl.chart.series import DataPoint
import datetime

# 1. Load the workbook
workbook = opxl.load_workbook("output.xlsx")
workbook.sheetnames

# 2. Activate the sheet
sheet = workbook.active

# 3. Define the row start and the column start for different portions of the table
row_start = 1
col_start = 1

# 4. Define Alignment and font styles
center_alignment = Alignment(horizontal="center", vertical="center")
left_alignment = Alignment(horizontal="left", vertical="center")
right_alignment = Alignment(horizontal="right", vertical="center")

bold_font = Font(bold=True)
italic_font = Font(italic=True)

# 5. Define borders
first_cell_border = Border(left=Side(style='thick'), right=Side(style='thin'), 
                          top=Side(style='thin'), bottom=Side(style='thin'))
last_cell_border = Border(left=Side(style='thin'), right=Side(style='thick'), 
                          top=Side(style='thin'), bottom=Side(style='thin'))
thin_box_border = Border(left=Side(style='thin'), right=Side(style='thin'),
                         top=Side(style='thin'), bottom=Side(style='thin'))
thick_box_border = Border(left=Side(style='thick'), right=Side(style='thick'), 
                     top=Side(style='thick'), bottom=Side(style='thick'))
thick_border = Border(left=Side(style='thin'), right=Side(style='thin'), 
                     top=Side(style='thick'), bottom=Side(style='thick'))
thick_bottom_border = Border(left=Side(style='thin'), right=Side(style='thin'), 
                     top=Side(style='thin'), bottom=Side(style='thick'))
thick_top_border = Border(left=Side(style='thin'), right=Side(style='thin'), 
                     top=Side(style='thick'), bottom=Side(style='thin'))
class_first_cell_border = Border(left=Side(style='thick'), right=Side(style='thin'), 
                          top=Side(style='thick'), bottom=Side(style='thin'))
class_last_cell_border = Border(left=Side(style='thick'), right=Side(style='thin'), 
                          top=Side(style='thick'), bottom=Side(style='thick'))

# 6. Define cell styles
header_style = NamedStyle(name="header_style")
header_style.font = Font(bold=True)
header_style.alignment = Alignment(horizontal="center", vertical="center")
header_style.border = Border(top=Side(border_style="thick"), bottom=Side(border_style="thick"), left=Side(border_style="thin"), right=Side(border_style="thin"))

first_cell_style = NamedStyle(name="first_cell_style")
first_cell_style.alignment = center_alignment
first_cell_style.border = first_cell_border

cell_style = NamedStyle(name="cell_style")
cell_style.alignment = center_alignment
cell_style.border = thin_box_border

last_cell_style = NamedStyle(name="last_cell_style")
last_cell_style.alignment = center_alignment
last_cell_style.border = last_cell_border

# 7. Define data format
currency_format = '#,##0.00 €'
percentage_format = '0.00%'

# 8. Define background colors
header_color = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')
footer_color = PatternFill(start_color='00FF00', end_color='00FF00', fill_type='solid')

# 9. Insert rows and columns as padding
rows_to_add = 2
cols_to_add = 1
sheet.insert_rows(idx=row_start, amount=rows_to_add)
sheet.insert_cols(idx=col_start, amount=cols_to_add)

# 10. Update the indexes for header, data and footer row
header_row_start = row_start + rows_to_add - 1
header_row_end = header_row_start + 1
footer_row_start = int(sheet.max_row) - 2
data_row_start = row_start + rows_to_add + 1
data_row_end = footer_row_start - 1
class_row_start = sheet.max_row + 5

col_start += cols_to_add
col_end = sheet.max_column

# 11. Update the data in the the Month row, format and merge cells 
sheet.cell(row=header_row_start, column=col_start).value = datetime.datetime.now().strftime("%b-%Y")
sheet.cell(row=header_row_start, column=col_start).fill = header_color
sheet.cell(row=header_row_start, column=col_start).alignment = center_alignment
sheet.cell(row=header_row_start, column=col_start).font = bold_font
sheet.cell(row=header_row_start, column=col_start).border = thick_box_border
sheet.merge_cells(start_row=header_row_start, start_column=col_start, end_row=header_row_start, end_column=col_end)

# 12. Adjust Column lengths
for column_cells in sheet.columns:
    new_column_length = max(len(str(cell.value)) for cell in column_cells)
    new_column_letter = (chr(64+(column_cells[0].column)))
    if sheet[new_column_letter][header_row_start].value == "COMMODITY":
        factor = 1   
    else:
        factor = 1.3
    if new_column_length > 0:
        sheet.column_dimensions[new_column_letter].width = new_column_length * factor

# 13. Style the header row
header_row = sheet[header_row_end][col_start-1:col_end]

for cell in header_row:
    cell.style = header_style
    # cell.fill = header_color
header_row[0].border = Border(left=Side(style='thick'), right=Side(style='thin'), 
                    top=Side(style='thick'), bottom=Side(style='thick'))
header_row[-1].border = Border(left=Side(style='thin'), right=Side(style='thick'), 
                    top=Side(style='thick'), bottom=Side(style='thick'))

# 14. Apply cell styles for individual cells
for ii in range(data_row_start, footer_row_start):
    for jj in range(col_start, col_end+1):
        if jj == col_start:
            sheet.cell(row=ii, column=jj).style = first_cell_style
        elif jj == col_start+1:
            sheet.cell(row=ii, column=jj).style = cell_style
            sheet.cell(row=ii, column=jj).alignment = Alignment(horizontal="left", vertical="center")
        elif jj == col_end:
             sheet.cell(row=ii, column=jj).style = last_cell_style
        else:
            sheet.cell(row=ii, column=jj).style = cell_style

# 15. Style the footer table
for r_idx in range(footer_row_start, sheet.max_row+1):
    rows_to_format = sheet[r_idx][col_start-1:col_end]
    for cell in rows_to_format:
        if r_idx == footer_row_start:
            cell.border = thick_top_border 
            rows_to_format[0].border = Border(left=Side(style='thick'), right=Side(style='thin'), 
                        top=Side(style='thick'), bottom=Side(style='thin'))
            rows_to_format[-1].border = Border(left=Side(style='thin'), right=Side(style='thick'), 
                        top=Side(style='thick'), bottom=Side(style='thin'))  
        elif r_idx == footer_row_start+1:
            cell.border = thin_box_border
            rows_to_format[0].border = Border(left=Side(style='thick'), right=Side(style='thin'), 
                        top=Side(style='thin'), bottom=Side(style='thin'))
            rows_to_format[-1].border = Border(left=Side(style='thin'), right=Side(style='thick'), 
                        top=Side(style='thin'), bottom=Side(style='thin'))
        else:
            cell.border = thick_bottom_border 
            rows_to_format[0].border = Border(left=Side(style='thick'), right=Side(style='thin'), 
                        top=Side(style='thin'), bottom=Side(style='thick'))          
            rows_to_format[-1].border = Border(left=Side(style='thin'), right=Side(style='thick'), 
                        top=Side(style='thin'), bottom=Side(style='thick'))                     
        cell.font = bold_font
        cell.fill = footer_color

In [2]:
# 16. Format the Amount column to be handled as currency
amount_column = sheet.max_column - 1
amount_column_letter = chr(64+amount_column)
for ii in range(row_start+1, sheet.max_row+1) :
    sheet.cell(row=ii, column=amount_column).number_format = currency_format
    sheet.cell(row=ii, column=amount_column).alignment = center_alignment

In [3]:
# 17. Add formula to the total, savings and percentage savings cell
total_expenses_cell = sheet.cell(row=footer_row_start, column=amount_column)
total_expenses_cell.value = f"=SUM({amount_column_letter + str(data_row_start)}:{amount_column_letter + str(total_expenses_cell.row-1)})"

salary_cell = sheet.cell(row=sheet.max_row-1, column=amount_column)
salary_cell.value = 2904.18

savings_cell = sheet.cell(row=sheet.max_row, column=amount_column)
savings_cell.value = f"={amount_column_letter+str(salary_cell.row)}" + '-' +  f"{amount_column_letter+str(total_expenses_cell.row)}"

percentage_savings_cell = sheet.cell(row=sheet.max_row, column=amount_column + 1)
percentage_savings_cell.value = f"={amount_column_letter+str(savings_cell.row)}" + '/' +  f"{amount_column_letter+str(salary_cell.row)}"
percentage_savings_cell.alignment = center_alignment
percentage_savings_cell.number_format = percentage_format


In [4]:
# 18. Populate and format the category classification table
categories = ["Savings (S)", "Rent (R)", "Investments (I)", "Loan (L)", "Groceries(G)", "Furnishing (F)", "Takeaway(T)", "Utilities(U)", "Miscellaneous(M)", "Travel (TR)", "Clothing (C)"]
class_row_end = class_row_start + len(categories) - 1
class_col_start = col_start + 1
class_col_end = class_col_start + 2
for column in sheet.iter_cols(min_row=class_row_start, max_row=class_row_end, min_col=class_col_start, max_col=class_col_end):
    for cell in column:
        if cell.column == class_col_start:
            cell.value = categories[cell.row-class_row_start]
        elif cell.column == class_col_start+1:
            if cell.row == class_row_start:
                cell.value = savings_cell.value 
            else:
                str_idx = categories[cell.row-class_row_start].find("(")
                criteria = categories[cell.row-class_row_start][str_idx:].lstrip("(").rstrip(")")
                sum_col = chr(64 + cell.column)
                range_col = chr(64 + sheet.max_column)
                cell.value = f'=SUMIF(${range_col}${data_row_start}:${range_col}${footer_row_start},"{criteria}",${sum_col}${data_row_start}:${sum_col}${footer_row_start})'
            cell.alignment = center_alignment
            cell.number_format = currency_format
        elif cell.column == class_col_end:
            col_letter = chr(64 + cell.column -1)
            cell.value = f"={col_letter}{cell.row}/{chr(64 + salary_cell.column)}{salary_cell.row}"
            cell.alignment = center_alignment
            cell.number_format = percentage_format

# for r_idx in [class_row_start, class_row_end]:
#     for ii in range(3):
#         if r_idx == class_row_start:
#             sheet.cell(row=r_idx, column=class_col_start + ii).border = Border(Side(style=''))
#         elif r_idx == class_row_end:
#             sheet.cell(row=r_idx, column=class_col_start + ii).border = thick_bottom_border

In [13]:
# 19. Plot a Pie-Chart for overview of expenses 
pie_chart = PieChart()
labels = Reference(worksheet=sheet, min_col=class_col_start, min_row=class_row_start, max_row=class_row_end)
data = Reference(worksheet=sheet, min_col=class_col_start+1,
                 min_row=class_row_start, max_row=class_row_end)

pie_chart.add_data(data, titles_from_data=True)
pie_chart.set_categories(labels)
pie_chart.title = datetime.datetime.now().strftime("%b-%Y")

chart_row = class_row_end + 5
chart_col = chr(64+col_start+1)
sheet.add_chart(pie_chart, f"{chart_col}{chart_row}")

pie_chart.height = 10
pie_chart.width = 10.5

# Cut the Savings slice out of the pie
slice = DataPoint(idx=0, explosion=10)
pie_chart.series[0].data_points = [slice]

C46
10 10.5


In [11]:
# 20. Delete the output file, if it already exists
if os.path.exists("./formatted_output.xlsx"):
    subprocess.run("del 'formatted_output.xlsx'")

In [12]:
# 21. Save the data in another excel sheet
workbook.save(filename="formatted_output.xlsx")